In [56]:
import pandas as pd
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.applications import MobileNet
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import numpy as np

In [113]:
df = pd.read_csv("./data.csv", index_col=None)

In [9]:
df

,text,label,start,end
0,1 марта 2006 года,Личные параметры,109,126
1,19 лет,Личные параметры,34,40
2,мужской,Личные параметры,47,54
3,русский,Личные параметры,72,79
4,затрудненное носовое дыхание,Симптомы,180,208
...,...,...,...,...
38862,продигиозан,Лекарство,9572,9583
38863,5-10,Значение,9604,9608
38864,нафталановая,Лекарство,9610,9622
38865,"эуфилиновая мази, кремы с 0,5% метотриксатом",Лекарство,9624,9668


In [12]:
import tensorflow as tf

print(tf.test.is_built_with_cuda())

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


False
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9224914029390426783
xla_global_id: -1
]


In [20]:
## Надо создать выборки и числовые вектора

In [114]:
x_train, x_test, y_train, y_test = train_test_split(df["text"],
                                                    data["label"],
                                                    test_size=0.1) 

In [115]:
y_train[0]

'Личные параметры'

In [116]:
label_cls = {}

i = 0
for item in y_train:
    if item not in label_cls.keys():
        label_cls[item] = i
        i += 1
        
label_index = []
for item in y_train:
    label_index.append(label_cls[item])

In [117]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
seq = tokenizer.texts_to_sequences(x_train)
word_index = tokenizer.word_index
data = pad_sequences(seq)

labels = to_categorical(np.asarray(label_index))

In [118]:
print(data.shape)
print(labels.shape)

(34980, 1255)
(34980, 14)


In [67]:
def build_model():
    model = Sequential()
    model.add(Embedding(34980, 128, input_length=1255))
    model.add(LSTM(128))
    model.add(Dropout(0.25))
    model.add(Dense(14, activation='softmax'))
    return model

In [71]:
model = build_model()
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.compile(loss='cross_entropy', optimizer='adam', metrics=['accuracy'])

In [72]:
history = model.fit(data,
                    labels,
                    epochs=5,
                    callbacks=[early_stopping])

Epoch 1/5
1094/1094 [==============================] - 804s 734ms/step - loss: 0.1391 - accuracy: 0.6217
Epoch 2/5
1094/1094 [==============================] - 757s 692ms/step - loss: 0.0601 - accuracy: 0.8652
Epoch 3/5
1094/1094 [==============================] - 730s 667ms/step - loss: 0.0364 - accuracy: 0.9216
Epoch 4/5
1094/1094 [==============================] - 739s 676ms/step - loss: 0.0273 - accuracy: 0.9405
Epoch 5/5
1094/1094 [==============================] - 745s 681ms/step - loss: 0.0234 - accuracy: 0.9461


In [76]:
model.save_weights("./first_model_w.h5")

In [77]:
model.save("./model.h5")

C:\Users\chern\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [82]:
item = data[0]
item = np.expand_dims(item, axis=0)

In [83]:
item.shape

(1, 1255)

In [86]:
pred = np.argmax(model.predict(item))

1/1 [==============================] - 0s 81ms/step


In [91]:
for item in label_cls.keys():
    if label_cls[item] == pred:
        print(item)

Значение


In [92]:
def get_prediction(model, data, labels):
    item = np.expand_dims(data, axis=0)
    pred = np.argmax(model.predict(item))
    for item in labels.keys():
        if labels[item] == pred:
            return item
    print("Нет решения")

In [133]:
text_t = tokenizer.sequences_to_texts(data)

In [137]:
len(text_t)

34980

In [139]:
for i in range(10):
    print(text_t[i] + ": " + get_prediction(model, data[i], label_cls))

1/1 [==============================] - 0s 81ms/step
58: Симптомы
1/1 [==============================] - 0s 82ms/step
115 мг: Дата
1/1 [==============================] - 0s 88ms/step
78 ударов в минуту: Симптомы
1/1 [==============================] - 0s 90ms/step
водц: Дата
1/1 [==============================] - 0s 87ms/step
правая и левая половины грудной клетки симметричны: Лекарство
1/1 [==============================] - 0s 88ms/step
анализ крови на гепатит с анти кс ат: Место
1/1 [==============================] - 0s 91ms/step
0 60 ме мл: Лекарство
1/1 [==============================] - 0s 102ms/step
2: Значение
1/1 [==============================] - 0s 93ms/step
при перкуссии отмечается смещение границ относительной тупости сердца левой на 2 см кнаружи от левой срединно ключичной линии nправая проходит по левому краю грудины nтоны сердца приглушены ритмичные чсс – 70 в мин nисследование сосудов артериальный пульс на лучевых артериях одинаков частота 70 уд в мин слабого наполнения и